In [3]:
import mysql.connector

In [4]:
mydb = mysql.connector.connect(
        host="127.0.0.1",
        user="root",
        password="1234567890",
        database="classicmodels"
    )

if mydb.is_connected():
    print("Successfully connected to MySQL database!")

Successfully connected to MySQL database!


In [5]:
mycursor = mydb.cursor()
mycursor.execute("""SELECT
    TABLE_SCHEMA,
    TABLE_NAME,
    COLUMN_NAME,
    DATA_TYPE,
    CHARACTER_MAXIMUM_LENGTH,
    IS_NULLABLE
    FROM
    INFORMATION_SCHEMA.COLUMNS
    WHERE
    TABLE_SCHEMA = 'classicmodels'
    ORDER BY
    TABLE_SCHEMA,
    TABLE_NAME,
    ORDINAL_POSITION;""")

myresult = mycursor.fetchall() # Fetches all rows
db_list = [[row[1], row[2], row[3], row[4], row[5]] for row in myresult]
    
print(db_list)
mycursor.close()

[['customers', 'customerNumber', 'int', None, 'NO'], ['customers', 'customerName', 'varchar', 50, 'NO'], ['customers', 'contactLastName', 'varchar', 50, 'NO'], ['customers', 'contactFirstName', 'varchar', 50, 'NO'], ['customers', 'phone', 'varchar', 50, 'NO'], ['customers', 'addressLine1', 'varchar', 50, 'NO'], ['customers', 'addressLine2', 'varchar', 50, 'YES'], ['customers', 'city', 'varchar', 50, 'NO'], ['customers', 'state', 'varchar', 50, 'YES'], ['customers', 'postalCode', 'varchar', 15, 'YES'], ['customers', 'country', 'varchar', 50, 'NO'], ['customers', 'salesRepEmployeeNumber', 'int', None, 'YES'], ['customers', 'creditLimit', 'decimal', None, 'YES'], ['employees', 'employeeNumber', 'int', None, 'NO'], ['employees', 'lastName', 'varchar', 50, 'NO'], ['employees', 'firstName', 'varchar', 50, 'NO'], ['employees', 'extension', 'varchar', 10, 'NO'], ['employees', 'email', 'varchar', 100, 'NO'], ['employees', 'officeCode', 'varchar', 10, 'NO'], ['employees', 'reportsTo', 'int', Non

True

In [6]:
from collections import defaultdict

result = defaultdict(list)

for item in db_list:
    key = item[0]
    value = item[1:]
    result[key].append(value)

result_dict = dict(result)

print(result_dict)

{'customers': [['customerNumber', 'int', None, 'NO'], ['customerName', 'varchar', 50, 'NO'], ['contactLastName', 'varchar', 50, 'NO'], ['contactFirstName', 'varchar', 50, 'NO'], ['phone', 'varchar', 50, 'NO'], ['addressLine1', 'varchar', 50, 'NO'], ['addressLine2', 'varchar', 50, 'YES'], ['city', 'varchar', 50, 'NO'], ['state', 'varchar', 50, 'YES'], ['postalCode', 'varchar', 15, 'YES'], ['country', 'varchar', 50, 'NO'], ['salesRepEmployeeNumber', 'int', None, 'YES'], ['creditLimit', 'decimal', None, 'YES']], 'employees': [['employeeNumber', 'int', None, 'NO'], ['lastName', 'varchar', 50, 'NO'], ['firstName', 'varchar', 50, 'NO'], ['extension', 'varchar', 10, 'NO'], ['email', 'varchar', 100, 'NO'], ['officeCode', 'varchar', 10, 'NO'], ['reportsTo', 'int', None, 'YES'], ['jobTitle', 'varchar', 50, 'NO']], 'offices': [['officeCode', 'varchar', 10, 'NO'], ['city', 'varchar', 50, 'NO'], ['phone', 'varchar', 50, 'NO'], ['addressLine1', 'varchar', 50, 'NO'], ['addressLine2', 'varchar', 50, '

In [7]:
openrouter_key="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"

In [8]:
nl_query="Show me the latest 10 customers that have purchased 1993 Mazda RX-7"
SAFE_ROW_LIMIT=100

In [ ]:
import requests
import json

response = requests.post(
  url="https://openrouter.ai/api/v1/chat/completions",
  headers={
    "Authorization": "Bearer <OPENROUTER_API_KEY>",
    "Content-Type": "application/json",
    
  },
  data=json.dumps({
    "model": "x-ai/grok-4-fast:free",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "What is in this image?"
          },
          {
            "type": "image_url",
            "image_url": {
              "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
            }
          }
        ]
      }
    ],
    
  })
)

In [10]:
import requests
import json
import os


url = "https://openrouter.ai/api/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {openrouter_key}",
    "Content-Type": "application/json",
}

payload = {
    "model": "x-ai/grok-4-fast:free",
    "messages": [
        {"role": "user", "content": f"""
    You are a SQL generation assistant for MySQL.
    DB schema (showing tables and columns): {result_dict}

    User request: {nl_query}

    Generate a safe, read-only MySQL SELECT query that answers the request.
    - Use correct column/table names from the schema.
    - Do not run updates or deletes.
    - If the user didn't ask for sorting/limit, apply LIMIT {SAFE_ROW_LIMIT}.
    - Return only the SQL, no backticks or extra explanation.
    """}
    ]
}

try:
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    response.raise_for_status()
    response_data = response.json()

    model_reply = response_data["choices"][0]["message"]["content"]
    print("Model Reply:")
    print(model_reply)

except requests.exceptions.RequestException as e:
    print(f"An error occurred during the API request: {e}")
    if hasattr(e, 'response') and e.response is not None:
        print(f"Response content: {e.response.text}")
except json.JSONDecodeError:
    print("Failed to decode JSON response.")
    print(f"Raw response content: {response.text}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Model Reply:
SELECT c.customerNumber, c.customerName
FROM customers c
JOIN (
    SELECT o.customerNumber, MAX(o.orderDate) AS latest_date
    FROM orders o
    JOIN orderdetails od ON o.orderNumber = od.orderNumber
    JOIN products p ON od.productCode = p.productCode
    WHERE p.productName = '1993 Mazda RX-7'
    GROUP BY o.customerNumber
) latest ON c.customerNumber = latest.customerNumber
ORDER BY latest_date DESC
LIMIT 10;


In [9]:
import requests
import json
import os


url = "https://openrouter.ai/api/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {openrouter_key}",
    "Content-Type": "application/json",
}

payload = {
    "model": "openrouter/sonoma-sky-alpha",
    "messages": [
        {"role": "user", "content": f"""
    You are a SQL generation assistant for MySQL.
    DB schema (showing tables and columns): {result_dict}

    User request: {nl_query}

    Generate a safe, read-only MySQL SELECT query that answers the request.
    - Use correct column/table names from the schema.
    - Do not run updates or deletes.
    - If the user didn't ask for sorting/limit, apply LIMIT {SAFE_ROW_LIMIT}.
    - Return only the SQL, no backticks or extra explanation.
    """}
    ]
}

try:
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    response.raise_for_status()
    response_data = response.json()

    model_reply = response_data["choices"][0]["message"]["content"]
    print("Model Reply:")
    print(model_reply)

except requests.exceptions.RequestException as e:
    print(f"An error occurred during the API request: {e}")
    if hasattr(e, 'response') and e.response is not None:
        print(f"Response content: {e.response.text}")
except json.JSONDecodeError:
    print("Failed to decode JSON response.")
    print(f"Raw response content: {response.text}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

An error occurred during the API request: 404 Client Error: Not Found for url: https://openrouter.ai/api/v1/chat/completions
Response content: {"error":{"message":"No endpoints found for openrouter/sonoma-sky-alpha.","code":404},"user_id":"user_30iby85ZmKpR1PJ7miumhTt0Wdn"}


In [67]:
mycursor = mydb.cursor()
try:
    mycursor.execute(model_reply)

    myresult = mycursor.fetchall() # Fetches all rows
    for row in myresult:
        print(row)

except Exception as e:
    print(f"Invalid query- {e}")
mycursor.close()

(382, 'Salzburg Collectables', 'Pipps', 'Georg ', '6562-9555', 'Salzburg', 'Austria', datetime.date(2005, 5, 17))
(145, 'Danish Wholesale Imports', 'Petersen', 'Jytte ', '31 12 3555', 'Kobenhavn', 'Denmark', datetime.date(2005, 4, 15))
(124, 'Mini Gifts Distributors Ltd.', 'Nelson', 'Susan', '4155551450', 'San Rafael', 'USA', datetime.date(2005, 3, 4))
(186, 'Toys of Finland, Co.', 'Karttunen', 'Matti', '90-224 8555', 'Helsinki', 'Finland', datetime.date(2005, 2, 9))
(320, 'Mini Creations Ltd.', 'Huang', 'Wing', '5085559555', 'New Bedford', 'USA', datetime.date(2005, 1, 7))
(250, 'Lyon Souveniers', 'Da Silva', 'Daniel', '+33 1 46 62 7555', 'Paris', 'France', datetime.date(2004, 12, 9))
(114, 'Australian Collectors, Co.', 'Ferguson', 'Peter', '03 9520 4555', 'Melbourne', 'Australia', datetime.date(2004, 11, 24))
(486, 'Motor Mint Distributors Inc.', 'Salazar', 'Rosa', '2155559857', 'Philadelphia', 'USA', datetime.date(2004, 11, 17))
(462, 'FunGiftIdeas.com', 'Benitez', 'Violeta', '50855

True

In [68]:
print(myresult)

[(382, 'Salzburg Collectables', 'Pipps', 'Georg ', '6562-9555', 'Salzburg', 'Austria', datetime.date(2005, 5, 17)), (145, 'Danish Wholesale Imports', 'Petersen', 'Jytte ', '31 12 3555', 'Kobenhavn', 'Denmark', datetime.date(2005, 4, 15)), (124, 'Mini Gifts Distributors Ltd.', 'Nelson', 'Susan', '4155551450', 'San Rafael', 'USA', datetime.date(2005, 3, 4)), (186, 'Toys of Finland, Co.', 'Karttunen', 'Matti', '90-224 8555', 'Helsinki', 'Finland', datetime.date(2005, 2, 9)), (320, 'Mini Creations Ltd.', 'Huang', 'Wing', '5085559555', 'New Bedford', 'USA', datetime.date(2005, 1, 7)), (250, 'Lyon Souveniers', 'Da Silva', 'Daniel', '+33 1 46 62 7555', 'Paris', 'France', datetime.date(2004, 12, 9)), (114, 'Australian Collectors, Co.', 'Ferguson', 'Peter', '03 9520 4555', 'Melbourne', 'Australia', datetime.date(2004, 11, 24)), (486, 'Motor Mint Distributors Inc.', 'Salazar', 'Rosa', '2155559857', 'Philadelphia', 'USA', datetime.date(2004, 11, 17)), (462, 'FunGiftIdeas.com', 'Benitez', 'Violeta

In [ ]:
import requests
import json
import os


url = "https://openrouter.ai/api/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {openrouter_key}",
    "Content-Type": "application/json",
}

payload = {
    "model": "openrouter/sonoma-sky-alpha",
    "messages": [
        {"role": "user", "content": f"""
    You are a SQL generation assistant for MySQL. Answer the question from the result ouputted from a previous SQL query. Explain it in a concise direct manner.

    Question: {nl_query}

    Previous Output- {myresult}

    Previous Query- {model_reply}
    """}
    ]
}

try:
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    response.raise_for_status()
    response_data = response.json()

    model_answer = response_data["choices"][0]["message"]["content"]
    print("Model Reply:")
    print(model_answer)

except requests.exceptions.RequestException as e:
    print(f"An error occurred during the API request: {e}")
    if hasattr(e, 'response') and e.response is not None:
        print(f"Response content: {e.response.text}")
except json.JSONDecodeError:
    print("Failed to decode JSON response.")
    print(f"Raw response content: {response.text}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Model Reply:
### Latest 10 Customers Who Purchased the 1993 Mazda RX-7

Below is the list of the 10 most recent customers who have purchased the "1993 Mazda RX-7", based on their latest purchase date. They are ordered from most recent to oldest. Each entry includes the customer ID, company name, contact details, phone, city, country, and the date of their most recent purchase of this product.

1. **Customer ID: 382**  
   Company: Salzburg Collectables  
   Contact: Georg Pipps  
   Phone: 6562-9555  
   City: Salzburg  
   Country: Austria  
   Last Purchase Date: 2005-05-17

2. **Customer ID: 145**  
   Company: Danish Wholesale Imports  
   Contact: Jytte Petersen  
   Phone: 31 12 3555  
   City: Kobenhavn  
   Country: Denmark  
   Last Purchase Date: 2005-04-15

3. **Customer ID: 124**  
   Company: Mini Gifts Distributors Ltd.  
   Contact: Susan Nelson  
   Phone: 4155551450  
   City: San Rafael  
   Country: USA  
   Last Purchase Date: 2005-03-04

4. **Customer ID: 186**  
 

In [71]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the API key
api_key = os.getenv("openrouter_key")

# Now you can use api_key in your application
print(f"Your API Key: {openrouter_key}")

Your API Key: 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


def fibonacci(n):
    a, b = 0, 1
    for _ in range(n):
        print(a, end=" ")
        a, b = b, a + b
